In [1]:
!pip install sccoda pertpy

  Using cached scCODA-0.1.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pertpy-0.9.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached tensorflow_probability-0.24.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached arviz-0.19.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached decoupler-1.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached lamin_utils-0.13.4-py2.py3-none-any.whl.metadata (980 bytes)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached PubChemPy-1.0.4-py3-none-any.whl
  Using cached scikit_misc-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
  Using cached scvi_tools-1.1.6.post2-py3-none-any.whl.metadata (18 kB)
  Using cached dm_tree-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached xarray-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached h5netcdf-1.3.0-py3-none-any.whl.metadata (13 kB)
  Using cached xarray_einstats-0.8.0-py3-none-any.wh

In [2]:
!pip install toytree arviz ete3

  Using cached toytree-3.0.5-py3-none-any.whl.metadata (3.6 kB)
  Using cached ete3-3.1.3-py3-none-any.whl
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached toyplot-1.0.3-py3-none-any.whl
  Using cached custom_inherit-2.4.1-py3-none-any.whl.metadata (838 bytes)
  Using cached pypng-0.20220715.0-py3-none-any.whl.metadata (13 kB)
  Using cached reportlab-4.2.2-py3-none-any.whl.metadata (1.4 kB)
Using cached toytree-3.0.5-py3-none-any.whl (382 kB)
Using cached loguru-0.7.2-py3-none-any.whl (62 kB)
Using cached custom_inherit-2.4.1-py3-none-any.whl (15 kB)
Using cached pypng-0.20220715.0-py3-none-any.whl (58 kB)
Using cached reportlab-4.2.2-py3-none-any.whl (1.9 MB)


In [3]:
import warnings

import pandas as pd

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

import scanpy as sc
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import altair as alt
import pertpy as pt

import itertools
import sys 

2024-09-22 18:31:00.243853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Dataset Background**
1. 18 Ulcerative Collitis Patients
2. 12 Healthy Indivudals

In [4]:
data = pt.data.smillie_2019()

**Pull Study Design from Original Paper**

In [5]:
study_file_path    = 'smillie_study_design.xlsx'
sheet_names        = pd.ExcelFile(study_file_path).sheet_names
study_design       = pd.read_excel(study_file_path, sheet_name = sheet_names[0])

In [6]:
unique_metadata   = {}
for col in data.obs:
    unique_metadata[col] = data.obs[col].unique()

**First Task**
1. Is there a connection between smokers and ulcerative collitis?
2. HC = Healthy Controls, UC  = Ulcerative Colitis, Colitis = Colities

In [7]:
meta_data       = study_design[['Disease', 'Smoking']]
meta_data.index = study_design['Subject ID']

diseased_ids    = meta_data[(meta_data['Disease'] == 'Colitis') | (meta_data['Disease'] == 'UC')].index
healthy_ids     = meta_data[~meta_data.index.isin(diseased_ids)].index

diseased_ids, healthy_ids = list(diseased_ids), list(healthy_ids)

**Pull the Unique Molecular Identifiers (UMIs) for the Diseased and Healthy Samples**

In [8]:
diseased_umis = data.obs[data.obs['Subject'].isin(diseased_ids)].index
healthy_umis  = data.obs[data.obs['Subject'].isin(healthy_ids)].index

In [44]:
diseased_umis

Index(['N7.EpiA.AAGGCTACCCTTTA', 'N7.EpiA.AAGGTGCTACGGAG',
       'N7.EpiA.AAGTAACTTGCTTT', 'N7.EpiA.ACAATAACCCTCAC',
       'N7.EpiA.ACAGTTCTTCTACT', 'N7.EpiA.ACGGTCCTGTACGT',
       'N7.EpiA.AGACTCGAAAGGGC', 'N7.EpiA.ATAAGTACAGATCC',
       'N7.EpiA.ATCTGTTGCTCTAT', 'N7.EpiA.ATGATATGTGGTTG',
       ...
       'N539.LPB.TTAGGCAGTAAGAGAG', 'N539.LPB.TTCGAAGCACGAAACG',
       'N539.LPB.TTCGGTCTCATTGCGA', 'N539.LPB.TTGCGTCCAATCGGTT',
       'N539.LPB.TTGGCAACATGTCGAT', 'N539.LPB.TTTATGCTCGGCTTGG',
       'N539.LPB.TTTGCGCCACGAGGTA', 'N539.LPB.TTTGGTTCACAGTCGC',
       'N539.LPB.TTTGTCAAGCTAGTGG', 'N539.LPB.TTTGTCAAGTGTTGAA'],
      dtype='object', length=244978)

**Data.X attribute appears to have the raw counts**
1. COnfirm this 09/20/2024

In [9]:
plot = False
if plot:
    sns.displot(data.X[:, :30].toarray())

In [10]:
data.layers['raw_counts'] = data.X.copy()

In [18]:
total_reads = 10000
sc.pp.normalize_total(data, total_reads)
sc.pp.log1p(data)


In [32]:
sc.pp.highly_variable_genes(data, max_mean = 3)

In [34]:
data_highvar = data[:, data.var.highly_variable]

In [40]:
sc.pp.pca(data_highvar)

In [41]:
sc.pp.neighbors(data_highvar)
sc.tl.umap(data_highvar)

In [58]:
data_highvar.obs.Cluster.unique()

['Plasma', 'CD8+ IELs', 'Tregs', 'Cycling T', 'CD8+ IL17+', ..., 'Best4+ Enterocytes', 'Secretory TA', 'Tuft', 'Enterocytes', 'Stem']
Length: 51
Categories (51, object): ['Best4+ Enterocytes', 'CD4+ Activated Fos-hi', 'CD4+ Activated Fos-lo', 'CD4+ Memory', ..., 'WNT2B+ Fos-lo 1', 'WNT2B+ Fos-lo 2', 'WNT5B+ 1', 'WNT5B+ 2']

In [59]:
sc.pl.umap(data_highvar, color = ['Health', 'Location', 'Cluster'], wspace = 0.5)